In [3]:
import os, sys
sys.path.append("/home/pqbas/dl")

In [5]:
import torch
from cnns.model_definition import resnet
from cnns.utils import model_builder, initialize_weights

# hyperparameters definition
NUM_CLASSES, IMG_SIZE, BATCH_SIZE = 10, 224, 32

# model build and weight initialization
model = model_builder(resnet)
model.apply(initialize_weights)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): residual_block(
    (F): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU()
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (I): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    )
  )
  (3): residual_block(
    (F): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (I): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
    )
  )
  (4): residual_block(
    (F): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b

In [9]:
if torch.cuda.device_count() > 1:
    print("Let's to train in ", torch.cuda.device_count(),"GPUs!")
    model = torch.nn.DataParallel(model)
else:
    print("You just have one GPU")

You just have one GPU


In [ ]:
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from cnns.utils import get_dataset, imshow, get_default_device, create_directory, save_model, train
import torch.optim as optim

data_transforms = [
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # Scales data into [0,1]
    transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1]
]

trainloader, testloader = get_dataset(dataset_name='cifar10', 
                                      transform=data_transforms, 
                                      batchsize = BATCH_SIZE)
imshow(testloader)

In [ ]:
# device = get_default_device()
device = "cuda"

trained_model = train(net = model,
                      epochs = 4,
                      trainloader = trainloader,
                      criterion = nn.CrossEntropyLoss(),
                      optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
                      device = device,
                      every_n_epochs = 2)